In [1]:
from ucimlrepo import fetch_ucirepo 
import numpy as np
from itertools import combinations
import pandas as pd
from sklearn.preprocessing import StandardScaler


# Fetch dataset
# wine = fetch_ucirepo(id=109)
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# Data (features and targets as pandas DataFrames)
# X = wine.data.features
# y = wine.data.targets
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

In [2]:
print(X.shape)
print(y.shape)


(569, 30)
(569, 1)


In [4]:
X.describe()

,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,fractal_dimension1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [17]:
y.describe()

,Diagnosis
count,569
unique,2
top,B
freq,357


In [24]:
from metric_learn import LMNN, NCA, ITML, ITML_Supervised
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [25]:
# Encode labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
# Create pairs of samples and corresponding similarity labels
def create_pairs(X, y):
    pairs = []
    labels = []

    # Iterate through all pairs of indices
    for i, j in combinations(range(len(y)), 2):
        pairs.append([X[i], X[j]])
        # Label 1 if they belong to the same class, -1 otherwise
        labels.append(1 if y[i] == y[j] else -1)
    
    return np.array(pairs), np.array(labels)

def create_pairs_iloc(X, y):
    pairs = []
    labels = []

    # Iterate through all pairs of indices
    for i, j in combinations(range(len(y)), 2):
        pairs.append([X.iloc[i], X.iloc[j]])
        # Label 1 if they belong to the same class, -1 otherwise
        labels.append(1 if y[i] == y[j] else -1)
    
    return np.array(pairs), np.array(labels)

In [26]:
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [27]:
# Train a k-NN classifier on the transformed data
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy knn: {accuracy}')

Accuracy knn: 0.9385964912280702


In [31]:
lmnn = LMNN(k=3, learn_rate=1e-6)
lmnn.fit(X_train, y_train)
X_train_lmnn = lmnn.transform(X_train)
X_test_lmnn = lmnn.transform(X_test)
knn.fit(X_train_lmnn, y_train)
y_pred = knn.predict(X_test_lmnn)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy lmnn: {accuracy}')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/metric_learn/lmnn.py:140: FutureWarning: "num_chunks" parameter has been renamed to "n_chunks". It has been deprecated in version 0.6.3 and will be removed in 0.7.0
  warnings.warn('"num_chunks" parameter has been renamed to'


Accuracy lmnn: 0.956140350877193


In [32]:
nca = NCA(max_iter=1000)
nca.fit(X_train, y_train)
X_train_nca = nca.transform(X_train)
X_test_nca = nca.transform(X_test)
knn.fit(X_train_nca, y_train)
y_pred = knn.predict(X_test_nca)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy nca: {accuracy}')

Accuracy nca: 0.956140350877193


In [37]:
itml = ITML_Supervised(max_iter=1000, n_constraints=3000)
itml.fit(X_train, y_train)
X_train_itml = itml.transform(X_train)
X_test_itml = itml.transform(X_test)
knn.fit(X_train_itml, y_train)
y_pred = knn.predict(X_test_itml)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy itml: {accuracy}')

Accuracy itml: 0.9473684210526315


In [ ]:
# Split the dataset
n_splits = 1
# Scale features
scaler = StandardScaler()

accuracies_knn = []
accuracies_knn_scaled = []
accuracies_lmnn = []
accuracies_nca = []
accuracies_itml = []

for n in range(n_splits):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    y_train = np.ravel(y_train)  # Convert to shape (n_samples,)
    y_test = np.ravel(y_test)
    # Generate pairs and labels for the training data
    pairs_train, labels_train = create_pairs_iloc(X_train, y_train)

    # LMNN
    lmnn = LMNN(k=3, learn_rate=1e-6)
    lmnn.fit(X_train, y_train)
    X_train_lmnn = lmnn.transform(X_train)
    X_test_lmnn = lmnn.transform(X_test)

    # NCA
    nca = NCA(max_iter=1000)
    nca.fit(X_train, y_train)
    X_train_nca = nca.transform(X_train)
    X_test_nca = nca.transform(X_test)

    # ITML
    itml = ITML()
    itml.fit(pairs_train, labels_train)
    # Transform the training and testing data (not pairs anymore)
    X_train_itml = itml.transform(X_train)
    X_test_itml = itml.transform(X_test)

    # Train a k-NN classifier on the transformed data
    knn = KNeighborsClassifier(n_neighbors=1)

    # Evaluate k-NN on original data
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies_knn.append(acc)

    knn.fit(X_train_scaled, y_train)
    y_pred_scaled = knn.predict(X_test_scaled)
    acc_scaled = accuracy_score(y_test, y_pred_scaled)
    accuracies_knn_scaled.append(acc_scaled)

    # Evaluate LMNN
    knn.fit(X_train_lmnn, y_train)
    y_pred_lmnn = knn.predict(X_test_lmnn)
    lmnn_acc = accuracy_score(y_test, y_pred_lmnn)
    accuracies_lmnn.append(lmnn_acc)

    # Evaluate NCA
    knn.fit(X_train_nca, y_train)
    y_pred_nca = knn.predict(X_test_nca)
    nca_acc = accuracy_score(y_test, y_pred_nca)
    accuracies_nca.append(nca_acc)

    # Evaluate ITML
    knn.fit(X_train_itml, y_train)
    y_pred_itml = knn.predict(X_test_itml)
    itml_acc = accuracy_score(y_test, y_pred_itml)
    accuracies_itml.append(itml_acc)



KeyboardInterrupt: 

In [49]:
print('k-NN (original data):', np.mean(accuracies_knn))
print('k-NN (scaled data):', np.mean(accuracies_knn_scaled))
print('LMNN:', np.mean(accuracies_lmnn))
print('NCA:', np.mean(accuracies_nca))
print('ITML:', np.mean(accuracies_itml))


k-NN (original data): 0.935672514619883
k-NN (scaled data): 0.9532163742690059
LMNN: 0.9649122807017544
NCA: 0.9239766081871345
ITML: 0.9532163742690059


In [37]:
# Fetch dataset
rtiot = fetch_ucirepo(id=942)

In [40]:
print(rtiot.keys())  # Should give something like: dict_keys(['data'])
print(rtiot['data'].keys())  # Should give: dict_keys(['ids', 'features'])


dict_keys(['data', 'metadata', 'variables'])
dict_keys(['ids', 'features', 'targets', 'original', 'headers'])


In [41]:
features_df = rtiot['data']['features']
print(features_df.head())  # Preview the first few rows


   id.orig_p  id.resp_p proto service  flow_duration  fwd_pkts_tot  \
0      38667       1883   tcp    mqtt      32.011598             9   
1      51143       1883   tcp    mqtt      31.883584             9   
2      44761       1883   tcp    mqtt      32.124053             9   
3      60893       1883   tcp    mqtt      31.961063             9   
4      51087       1883   tcp    mqtt      31.902362             9   

   bwd_pkts_tot  fwd_data_pkts_tot  bwd_data_pkts_tot  fwd_pkts_per_sec  ...  \
0             5                  3                  3          0.281148  ...   
1             5                  3                  3          0.282277  ...   
2             5                  3                  3          0.280164  ...   
3             5                  3                  3          0.281593  ...   
4             5                  3                  3          0.282111  ...   

    active.avg  active.std     idle.min     idle.max     idle.tot  \
0  2282414.913         0.0  2

In [33]:
# Data (features and targets as pandas DataFrames)
X = rtiot.data.features
y = rtiot.data.targets

In [34]:
# Split the dataset
n_splits = 1
# Scale features
scaler = StandardScaler()

accuracies_knn = []
accuracies_knn_scaled = []
accuracies_lmnn = []
accuracies_nca = []
accuracies_itml = []

for n in range(n_splits):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    y_train = np.ravel(y_train)  # Convert to shape (n_samples,)
    y_test = np.ravel(y_test)
    # Generate pairs and labels for the training data
    pairs_train, labels_train = create_pairs_iloc(X_train, y_train)

    # LMNN
    lmnn = LMNN(k=3, learn_rate=1e-6)
    lmnn.fit(X_train, y_train)
    X_train_lmnn = lmnn.transform(X_train)
    X_test_lmnn = lmnn.transform(X_test)

    # NCA
    nca = NCA(max_iter=1000)
    nca.fit(X_train, y_train)
    X_train_nca = nca.transform(X_train)
    X_test_nca = nca.transform(X_test)

    # ITML
    itml = ITML()
    itml.fit(pairs_train, labels_train)
    # Transform the training and testing data (not pairs anymore)
    X_train_itml = itml.transform(X_train)
    X_test_itml = itml.transform(X_test)

    # Train a k-NN classifier on the transformed data
    knn = KNeighborsClassifier(n_neighbors=1)

    # Evaluate k-NN on original data
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies_knn.append(acc)

    knn.fit(X_train_scaled, y_train)
    y_pred_scaled = knn.predict(X_test_scaled)
    acc_scaled = accuracy_score(y_test, y_pred_scaled)
    accuracies_knn_scaled.append(acc_scaled)

    # Evaluate LMNN
    knn.fit(X_train_lmnn, y_train)
    y_pred_lmnn = knn.predict(X_test_lmnn)
    lmnn_acc = accuracy_score(y_test, y_pred_lmnn)
    accuracies_lmnn.append(lmnn_acc)

    # Evaluate NCA
    knn.fit(X_train_nca, y_train)
    y_pred_nca = knn.predict(X_test_nca)
    nca_acc = accuracy_score(y_test, y_pred_nca)
    accuracies_nca.append(nca_acc)

    # Evaluate ITML
    knn.fit(X_train_itml, y_train)
    y_pred_itml = knn.predict(X_test_itml)
    itml_acc = accuracy_score(y_test, y_pred_itml)
    accuracies_itml.append(itml_acc)

ValueError: could not convert string to float: 'tcp'